In [1]:
%matplotlib inline
import random
import pandas as ps
import numpy as np
from math import log
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
TRAIN_FILE = '/home/sergio/Data/titanic/train.csv'
TEST_FILE = '/home/sergio/Data/titanic/test.csv'

# Построение модели с максимальным набором признаков

## Подготовка данных

In [2]:
class Transformer:
    _title_mapping = {
        "mister": {"Capt", "Col", "Don", "Dr", "Jonkheer", "Major", "Mr", "Rev", "Sir"}, # Взрослые мужчины
        "missis": {"Mme", "Mrs", "Dona", "the Countess"}, # Взрослые женщины
        "miss": {"Lady", "Miss", "Mlle", "Ms"}, # Девочки и девушки
        "master": {"Master"} # Мальчики и юноши
    }
    
    def __init__(self, train, target, fulldata):
        self._age_maping = {(sex, cls): fulldata[(fulldata.Sex == sex) & (fulldata.Pclass == cls)].Age.median()
                                      for sex in ["male", "female"] for cls in range(1,4)}
        self._tickets_count = fulldata.groupby(["Ticket"])["PassengerId"].count()
        
        # TICKET SURVIVAL
        self._tickets_survival = {ticket: {"survived": 0, "perished": 0} for ticket in train.Ticket.drop_duplicates()}
        for ind, row in train.iterrows():
            if target[ind]:
                self._tickets_survival[row.Ticket]["survived"] += 1
            else:
                self._tickets_survival[row.Ticket]["perished"] += 1
        
        
        tmp = fulldata.copy()
        tmp.insert(len(tmp.columns), "Surname", [name.split(",")[0].strip() for name in tmp.Name])
        self._surnames_count = tmp.groupby(["Surname"])["PassengerId"].count()
        
        # SURNAME SURVIVAL
        tmp = train.copy()
        tmp.insert(len(tmp.columns), "Surname", [name.split(",")[0].strip() for name in tmp.Name])
        self._surnames_survival = {surname: {"survived": 0, "perished": 0} for surname in tmp.Surname.drop_duplicates()}
        for ind, row in tmp.iterrows():
            if target[ind]:
                self._surnames_survival[row.Surname]["survived"] += 1
            else:
                self._surnames_survival[row.Surname]["perished"] += 1
        
        # TICKET + SURNAME SURVIVAL
        tmp.insert(len(tmp.columns), "SurnameAndTicket", [(row.Surname, row.Ticket) for ind, row in tmp.iterrows()])
        self._surtick_survival = {surtick: {"survived": 0, "perished": 0}
                                  for surtick in tmp.SurnameAndTicket.drop_duplicates()}
        for ind, row in tmp.iterrows():
            if target[ind]:
                self._surtick_survival[row.SurnameAndTicket]["survived"] += 1
            else:
                self._surtick_survival[row.SurnameAndTicket]["perished"] += 1
        
        self._codes = {}
    
    def apply(self, data, target=None):
        data.Age = [self._age_maping[(row.Sex,row.Pclass)] if np.isnan(row.Age) else row.Age
                    for ind, row in data.iterrows()]
        data.Fare.fillna(0, inplace=True)
        data.Embarked.fillna("S", inplace=True)
        data.insert(len(data.columns), "HasCabin", [int(type(cabin) == str) for cabin in data.Cabin])
        data.insert(len(data.columns), "Title", [self.title(name) for name in data.Name])
        data.insert(len(data.columns), "SurnameCount", [self.surnameCount(name) for name in data.Name])
        data.insert(len(data.columns), "FarRelatives", [row.SurnameCount-row.SibSp-row.Parch for ind, row in data.iterrows()])
        data.insert(len(data.columns), "Cotravellers", [self._tickets_count[ticket] for ticket in data.Ticket])
        data.insert(len(data.columns), "FarePerTraveller", [row.Fare/row.Cotravellers for ind, row in data.iterrows()])
        data.insert(len(data.columns), "TicketSurvival",
                    [self.ticketSurvival(row.Ticket, target=(target[ind] if target is not None else None))
                     for ind, row in data.iterrows()])
        data.insert(len(data.columns), "SurnameSurvival",
                    [self.surnameSurvival(row.Name, target=(target[ind] if target is not None else None))
                     for ind, row in data.iterrows()])
        data.insert(len(data.columns), "SurtickSurvival",
                    [self.surtickSurvival(row.Name, row.Ticket, target=(target[ind] if target is not None else None))
                     for ind, row in data.iterrows()])
        data.Fare = [log(fare, 3) if fare != 0 else 0 for fare in data.Fare]
        data.FarePerTraveller = [log(fare, 3) if fare != 0 else 0 for fare in data.FarePerTraveller]
        data = self.oneHot(data, "Embarked")
        data = self.oneHot(data, "Pclass")
        data = self.encode(data, "Sex")
        data = self.oneHot(data, "Title")
        return data.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)

    def encode(self, data, column):
        values = data[column].drop_duplicates()
        if column in self._codes:
            mapping = self._codes[column]
            for val in values:
                if val not in mapping:
                    mapping[val] = len(mapping)
        else:
            mapping = {v: i for i, v in enumerate(sorted(values))}
            self._codes[column] = mapping
        data[column] = [mapping[v] for v in data[column]]
        return data
    
    def oneHot(self, data, column):
        values = data[column].drop_duplicates()
        for val in sorted(values):
            data.insert(len(data.columns), "{}_{}".format(column, val),
                        [1 if rowval == val else 0 for rowval in data[column]])
        return data.drop([column], axis=1)
    
    @classmethod
    def title(cls, name):
        title = name.split(",")[1].split(".")[0].strip()
        return next(key for key, value in cls._title_mapping.items() if title in value)

    def surnameCount(self, name):
        return self._surnames_count[name.split(",")[0].strip()]
    
    def ticketSurvival(self, ticket, target=None):
        try:
            info = self._tickets_survival[ticket]
        except KeyError:
            return 0
        index = lambda surv, per: surv - per
        if target is None:
            return index(info["survived"], info["perished"])
        elif target == 1:
            return index(info["survived"]-1, info["perished"])
        elif target == 0:
            return index(info["survived"], info["perished"]-1)
    
    def surnameSurvival(self, name, target=None):
        surname = name.split(",")[0].strip()
        try:
            info = self._surnames_survival[surname]
        except KeyError:
            return 0
        index = lambda surv, per: surv - per
        if target is None:
            return index(info["survived"], info["perished"])
        elif target == 1:
            return index(info["survived"]-1, info["perished"])
        elif target == 0:
            return index(info["survived"], info["perished"]-1)
    
    def surtickSurvival(self, name, ticket, target=None):
        surname = name.split(",")[0].strip()
        try:
            info = self._surtick_survival[(surname, ticket)]
        except KeyError:
            return 0
        index = lambda surv, per: surv - per
        if target is None:
            return index(info["survived"], info["perished"])
        elif target == 1:
            return index(info["survived"]-1, info["perished"])
        elif target == 0:
            return index(info["survived"], info["perished"]-1)

In [3]:
def get_data():
    data = ps.read_csv(TRAIN_FILE)
    test = ps.read_csv(TEST_FILE)
    train = data.drop(['Survived'], axis=1)
    target = data.Survived
    fulldata = ps.concat([train, test])
    trans = Transformer(train, target, fulldata)
    return trans.apply(train, target), target, trans.apply(test)

train, target, test = get_data()
train[:5].drop(["Embarked_C", "Embarked_Q", "Embarked_S", "Fare", "HasCabin", "FarePerTraveller"], axis=1)

,Sex,Age,SibSp,Parch,SurnameCount,FarRelatives,Cotravellers,TicketSurvival,SurnameSurvival,SurtickSurvival,Pclass_1,Pclass_2,Pclass_3,Title_master,Title_miss,Title_missis,Title_mister
0,1,22,1,0,2,1,1,0,-1,0,0,0,1,0,0,0,1
1,0,38,1,0,2,1,2,0,0,0,1,0,0,0,0,1,0
2,0,26,0,0,1,1,1,0,0,0,0,0,1,0,1,0,0
3,0,35,1,0,2,1,2,-1,-1,-1,1,0,0,0,0,1,0
4,1,35,0,0,2,2,1,0,1,0,0,0,1,0,0,0,1


In [4]:
test[:5].drop(["Embarked_C", "Embarked_Q", "Embarked_S"], axis=1)

,Sex,Age,SibSp,Parch,Fare,HasCabin,SurnameCount,FarRelatives,Cotravellers,FarePerTraveller,TicketSurvival,SurnameSurvival,SurtickSurvival,Pclass_1,Pclass_2,Pclass_3,Title_master,Title_miss,Title_missis,Title_mister
0,1,34.5,0,0,1.873145,0,5,5,1,1.873145,0,2,0,0,0,1,0,0,0,1
1,0,47.0,1,0,1.771244,0,1,0,1,1.771244,0,0,0,0,0,1,0,0,1,0
2,1,62.0,0,0,2.067004,0,1,1,1,2.067004,0,0,0,0,1,0,0,0,0,1
3,1,27.0,0,0,1.965210,0,1,1,1,1.965210,0,0,0,0,0,1,0,0,0,1
4,0,22.0,1,1,2.283410,0,2,0,2,1.652480,1,1,1,0,0,1,0,0,1,0


## Построение и оценка моделей

In [7]:
models = [
    RandomForestClassifier(n_estimators = 80, max_features='auto', criterion='entropy', max_depth=4),
    LogisticRegression(penalty='l1', tol=0.01)
]
for mdl in models:
    scores = cross_validation.cross_val_score(mdl, train, target, cv=3)
    print("{}: {:.4f} (+/- {:.4f})".format(mdl.__class__.__name__, scores.mean(), scores.std() * 2))

RandomForestClassifier: 0.8373 (+/- 0.0193)
LogisticRegression: 0.8316 (+/- 0.0334)


Экспортируем результаты:

In [19]:
for mdl in models:
    mdl.fit(train, target)
    result = ps.DataFrame()
    result.insert(0, "PassengerId", range(892, 1310))
    result.insert(1, "Survived", mdl.predict(test))
    result.to_csv("results/linkfeatures-v3-r2-{}.csv".format(mdl.__class__.__name__), index=False)

Данная модель даёт результат в 0.79904 методом Random Forest.